In [ ]:
pip install psycopg2-binary scikit-learn numpy

In [1]:
import os
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [8]:
# Path to the articles directory
ARTICLES_DIR = "D:/TrueLensSdgp/TrueLens/news_filtered_data/news_source_data/data/articles"

def load_articles(directory):
    """ Load articles from JSON files and filter out empty ones. """
    articles = []
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                with open(os.path.join(root, file), "r", encoding="utf-8") as f:
                    data = json.load(f)
                    text = " ".join(data["body_paragraphs"]).strip()
                    
                    # Skip empty articles
                    if text:
                        articles.append({
                            "url": data["url"],
                            "title": data["title"],
                            "ut": data["ut"],
                            "body_paragraphs": text
                        })
    
    return articles

# Test article loading
articles = load_articles(ARTICLES_DIR)
print(f"Loaded {len(articles)} articles.")


Loaded 0 articles.
